In [141]:
import pandas as pd
import os
from tqdm.auto import tqdm
import subprocess
from utils.functions import parse
import warnings
from gensim.models.word2vec import Word2Vec
import numpy as np
import nltk
import torch
import json
from numpyencoder import NumpyEncoder
from pydriller import Repository
import pickle
import shutil

warnings.filterwarnings("ignore")

# Get codes from D2A

In [77]:
file_list = os.listdir('data/D2A_func')
len(file_list)

6728

In [78]:
dfs = [] # an empty list to store the data frames
for file in tqdm(file_list):
    data = pd.read_json('data/D2A_func/'+file, lines=True) # read data frame from json file
    dfs.append(data) # append the data frame to the list

temp = pd.concat(dfs, ignore_index=True) 

In [79]:
temp

,id,label,bug_location,bug_info,bug_type,project,src,code,compiler_args,version_hash,prefix,suffix
0,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@249:1-262:2,"void ff_clear_fixed_vector(float *out, const A...",,290e7eb77bee5a54182fb3d5fb122c1e117190da,,
1,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_1,1,[10],"([-oo, +oo] + `in->pitch_lag`):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@248:1-260:2,"void ff_clear_fixed_vector(float *out, const A...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,b6b00675112b411fbb10e8b01be1d3ddc2574cf9,/*\n * adaptive and fixed codebook vector oper...,
2,ffmpeg_00254aed9cd491784527972397a55016f5c3f270_1,1,[44],"([0, +oo] × [-oo, +oo]):signed32 by call to `r...",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/twinvq.c@730:1-813:2,static void read_and_decode_spectrum(TwinConte...,-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,5ed68178225c82f6525f962b895bce682e9ad05c,/*\n * TwinVQ decoder\n * Copyright (c) 2009 V...,\nstatic int twin_decode_frame(AVCodecContext ...
3,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@2226:1-2345:2,static void vp8_decode_mb_row_no_filter(AVCode...,,ac4b32df71bd932838043a4838b86d11e169707f,,
4,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_1,1,[33],"([-oo, +oo] - 1):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@1702:1-1817:2,static void vp8_decode_mb_row_no_filter(AVCode...,-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,ce58ead40dd16f272ec1f127f04f69df67c7cd35,/*\n * VP8 compatible video decoder\n *\n * Co...,\nstatic void vp8_filter_mb_row(AVCodecContext...
...,...,...,...,...,...,...,...,...,...,...,...,...
6723,openssl_ffa54528d94a1a94cbb12a803ed2889a256e32...,0,[],NaN,DEAD_STORE,openssl,crypto/asn1/a_utctm.c@16:1-22:2,"int asn1_utctime_to_tm(struct tm *tm, const AS...",,3d0f1cb9fdd630c6c920bc97bf496538717e7705,,
6724,openssl_ffa54528d94a1a94cbb12a803ed2889a256e32...,1,[32],The value written to &i (type int) is never used.,DEAD_STORE,openssl,crypto/asn1/a_utctm.c@16:1-131:2,"int asn1_utctime_to_tm(struct tm *tm, const AS...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,a1099821f9937717f92464056d80f2e303a73a4d,/*\n * Copyright 1995-2017 The OpenSSL Project...,\nint ASN1_UTCTIME_check(const ASN1_UTCTIME *d...
6725,openssl_ffa7bc4ef5ec59f57a339daad0fe6983119c16...,1,[180],"([-oo, +oo] - [-oo, +oo]):signed64.",INTEGER_OVERFLOW_U5,openssl,apps/asn1pars.c@56:1-311:2,"int asn1parse_main(int argc, char **argv)\n{\n...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,4f090f76a412a0f69f85621468bd445ea6a65af6,/*\n * Copyright 1995-2018 The OpenSSL Project...,"\nstatic int do_generate(char *genstr, const c..."
6726,openssl_ffd8fb02e505c5267d9b46914d77a10a6a7c68...,0,[],NaN,BUFFER_OVERRUN_L4,openssl,apps/req.c@1181:1-1237:2,"static int add_DN_object(X509_NAME *n, char *t...",,7644a9aef8932ed4d1c3f25ed776c997702982be,,


In [80]:
temp.to_csv("Datasets for VELVET/d2a.csv", index=False)

# Create cpg

In [92]:
raw = pd.read_csv("Datasets for VELVET/d2a.csv")

In [82]:
proc = subprocess.Popen(["echo", "%BASEDIR%"], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
path_to_joern = out.decode('utf-8').split('\r')[0]

In [83]:
def to_files(data_frame: pd.DataFrame, out_path):
    for idx, row in tqdm(data_frame.iterrows()):
        extension = row.src.split('.')[-1][0]
        file_name = f"{idx}.{extension}"
        try:
            with open(out_path + file_name, 'w') as f:
                f.write(row.code)
        except UnicodeEncodeError as e:
            print(e)
            print(row.id)
            print(row.code)
            break;

In [84]:
to_files(raw, 'data/code/')

In [85]:
subprocess.run([path_to_joern + "/joern-parse.bat", "data/code"],
                                      stdout=subprocess.PIPE, text=True, check=True)

CompletedProcess(args=['C:\\Users\\Grazia\\Desktop\\Tesi\\StackOverflow_Vulnerabilities\\1-Vulnerability_Detection\\joern/joern-parse.bat', 'data/code'], returncode=0, stdout='\nC:\\Users\\Grazia\\Desktop\\Tesi\\StackOverflow_Vulnerabilities\\1-Vulnerability_Detection>java -cp "C:\\Users\\Grazia\\Desktop\\Tesi\\StackOverflow_Vulnerabilities\\1-Vulnerability_Detection\\joern/projects/extensions/joern-fuzzyc/build/libs/joern-fuzzyc.jar;C:\\Users\\Grazia\\Desktop\\Tesi\\StackOverflow_Vulnerabilities\\1-Vulnerability_Detection\\joern/projects/extensions/jpanlib/build/libs/jpanlib.jar;C:\\Users\\Grazia\\Desktop\\Tesi\\StackOverflow_Vulnerabilities\\1-Vulnerability_Detection\\joern/projects/octopus/lib/*" tools.parser.ParserMain -outformat csv -outdir parsed data/code \ndata\\code\\0.c\ndata\\code\\1.c\ndata\\code\\10.c\ndata\\code\\100.c\ndata\\code\\1000.c\ndata\\code\\1001.c\ndata\\code\\1002.c\ndata\\code\\1003.c\ndata\\code\\1004.c\ndata\\code\\1005.c\ndata\\code\\1006.c\ndata\\code\\10

# Embedding

In [93]:
raw = raw.rename(columns={'code':'tokens'})

raw.head()

,id,label,bug_location,bug_info,bug_type,project,src,tokens,compiler_args,version_hash,prefix,suffix
0,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@249:1-262:2,"void ff_clear_fixed_vector(float *out, const A...",NaN,290e7eb77bee5a54182fb3d5fb122c1e117190da,NaN,NaN
1,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_1,1,[10],"([-oo, +oo] + `in->pitch_lag`):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@248:1-260:2,"void ff_clear_fixed_vector(float *out, const A...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,b6b00675112b411fbb10e8b01be1d3ddc2574cf9,/*\n * adaptive and fixed codebook vector oper...,NaN
2,ffmpeg_00254aed9cd491784527972397a55016f5c3f270_1,1,[44],"([0, +oo] × [-oo, +oo]):signed32 by call to `r...",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/twinvq.c@730:1-813:2,static void read_and_decode_spectrum(TwinConte...,-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,5ed68178225c82f6525f962b895bce682e9ad05c,/*\n * TwinVQ decoder\n * Copyright (c) 2009 V...,\nstatic int twin_decode_frame(AVCodecContext ...
3,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@2226:1-2345:2,static void vp8_decode_mb_row_no_filter(AVCode...,NaN,ac4b32df71bd932838043a4838b86d11e169707f,NaN,NaN
4,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_1,1,[33],"([-oo, +oo] - 1):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@1702:1-1817:2,static void vp8_decode_mb_row_no_filter(AVCode...,-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,ce58ead40dd16f272ec1f127f04f69df67c7cd35,/*\n * VP8 compatible video decoder\n *\n * Co...,\nstatic void vp8_filter_mb_row(AVCodecContext...


In [94]:
for index in tqdm(range(raw.shape[0])):
    raw['tokens'][index] = parse.tokenizer(raw['tokens'][index])

In [96]:
raw.to_csv("Datasets for VELVET/d2a_tokenized.csv", index=False)

In [99]:
w2vmodel = Word2Vec(min_count=1)
w2vmodel.build_vocab(sentences=tqdm(raw.tokens))
w2vmodel.train(sentences=tqdm(raw.tokens), total_examples=w2vmodel.corpus_count, epochs=1)
print("Saving w2vmodel.")
w2vmodel.save(f"data/w2v/w2v_model")


Saving w2vmodel.


# Creating input files

In [102]:
EDGE_TYPES = {
    "CONTROLS": 0,
    "DECLARES": 1,
    "DEF": 2,
    "DOM": 3,
    "FLOWS_TO": 4,
    "IS_AST_PARENT": 5,
    "IS_CLASS_OF": 6,
    "IS_FILE_OF": 7,
    "IS_FUNCTION_OF_AST": 8,
    "IS_FUNCTION_OF_CFG": 9,
    "POST_DOM": 10,
    "REACHES": 11,
    "USE": 12,
}

TYPE_MAP = {
    "AndExpression": 1,
    "Sizeof": 2,
    "Identifier": 3,
    "ForInit": 4,
    "ReturnStatement": 5,
    "SizeofOperand": 6,
    "InclusiveOrExpression": 7,
    "PtrMemberAccess": 8,
    "AssignmentExpression": 9,
    "ParameterList": 10,
    "IdentifierDeclType": 11,
    "SizeofExpression": 12,
    "SwitchStatement": 13,
    "IncDec": 14,
    "Function": 15,
    "BitAndExpression": 16,
    "UnaryExpression": 17,
    "DoStatement": 18,
    "GotoStatement": 19,
    "Callee": 20,
    "OrExpression": 21,
    "ShiftExpression": 22,
    "Decl": 23,
    "CFGErrorNode": 24,
    "WhileStatement": 25,
    "InfiniteForNode": 26,
    "RelationalExpression": 27,
    "CFGExitNode": 28,
    "Condition": 29,
    "BreakStatement": 30,
    "CompoundStatement": 31,
    "UnaryOperator": 32,
    "CallExpression": 33,
    "CastExpression": 34,
    "ConditionalExpression": 35,
    "ArrayIndexing": 36,
    "PostIncDecOperationExpression": 37,
    "Label": 38,
    "ArgumentList": 39,
    "EqualityExpression": 40,
    "ReturnType": 41,
    "Parameter": 42,
    "Argument": 43,
    "Symbol": 44,
    "ParameterType": 45,
    "Statement": 46,
    "AdditiveExpression": 47,
    "PrimaryExpression": 48,
    "DeclStmt": 49,
    "CastTarget": 50,
    "IdentifierDeclStatement": 51,
    "IdentifierDecl": 52,
    "CFGEntryNode": 53,
    "TryStatement": 54,
    "Expression": 55,
    "ExclusiveOrExpression": 56,
    "ClassDef": 57,
    "File": 58,
    "UnaryOperationExpression": 59,
    "ClassDefStatement": 60,
    "FunctionDef": 61,
    "IfStatement": 62,
    "MultiplicativeExpression": 63,
    "ContinueStatement": 64,
    "MemberAccess": 65,
    "ExpressionStatement": 66,
    "ForStatement": 67,
    "InitializerList": 68,
    "ElseStatement": 69,
    'ThrowStatement': 70,
    'CFGExceptionNode': 71,
    'CatchStatement': 72,
    'CatchList': 73
}

TYPE_MAP_OH = np.eye(len(TYPE_MAP))

In [103]:
def create_input_from_cpg(filepath, flaw_loc, w2v, etypemap = EDGE_TYPES):
    try:
        nodes = pd.read_csv(filepath+"/nodes.csv", sep="\t")
        edges = pd.read_csv(filepath+"/edges.csv", sep="\t")
    except Exception as E:
        print(E)
        return [None, None]
    if len(nodes) == 0 or len(edges) == 0:
        print("Empty node / edge CSV")
        return [None, None]
    n, e = format_node_edges(nodes, edges)
    e = e[e.type != "IS_FILE_OF"]
    e = e[e.type.isin(etypemap)]
    
    node_id_dict = (
    n.reset_index(drop=True)
    .reset_index()[["key", "index"]]
    .set_index("key")
    .to_dict()["index"]
    )
    e = e[(e.start.isin(n.key)) & (e.end.isin(n.key))].copy()
    n.key = n.key.apply(lambda x: node_id_dict[x])
    e.start = e.start.apply(lambda x: node_id_dict[x])
    e.end = e.end.apply(lambda x: node_id_dict[x])
    
    n.code = n.code.fillna("")
    n.code = n.code.apply(embed_code, w2v=w2v).to_list()
    n.type = n.type.apply(one_hot_encode_type)
    n = n[n.type != -1]
    
    src = e["start"].to_numpy()
    dst = e["end"].to_numpy()
    nnodes = len(n)
    nfeat = torch.tensor([list(i.type) + list(i.code) for i in n.itertuples()]).float()
    etype = torch.tensor([etypemap[i] for i in e.type]).int()
    return n, e
    
    
    
def format_node_edges(n, e):
    """Format node and edges into appropriate input form."""
    nodes = n.copy()
    edges = e.copy()
    nodes.key -= 1
    edges.start -= 1
    edges.end -= 1
    node_key_type_map = nodes[["key", "type"]].set_index("key").to_dict()["type"]
    node_key_code_map = nodes[["key", "code"]].set_index("key").to_dict()["code"]
    edges["src"] = edges.start.apply(lambda x: node_key_type_map[x])
    edges["dest"] = edges.end.apply(lambda x: node_key_type_map[x])
    edges["src_feat"] = edges.start.apply(lambda x: node_key_code_map[x])
    edges["dest_feat"] = edges.end.apply(lambda x: node_key_code_map[x])
    return nodes, edges

def one_hot_encode_type(type_):
    try:
        return TYPE_MAP_OH[TYPE_MAP[type_] - 1].tolist()
    except:
        return -1
    
def embed_code(code: str, w2v):
    """Embed code using given word2vec model by averaging code embeddings."""
    code = nltk.word_tokenize(code.strip())
    if len(code) == 0:
        return np.zeros(100)
    wvecs = []
    for word in code:
        try:
            wvecs.append(w2v.wv[word])
        except:
            wvecs.append(np.zeros(100))
    return np.array(sum(np.array(wvecs)) / len(code), dtype="float32")

In [127]:
base_dir = "parsed/data/code/"
index_ben = 0
index_vul = 0
w2vmodel = Word2Vec.load("data/w2v/w2v_model")

for file in tqdm(os.listdir('data/code')):
    filename = file.split('.')[0]
    flaw_loc = raw.iloc[int(filename), 2]
    n, e = create_input_from_cpg(base_dir+file, flaw_loc, w2vmodel)
    
    if n is not None and e is not None:
        edges_list = list()
        targets = list()
        map_line = list()

        for index_edge in range(e.shape[0]):
            if index_edge in e.index:
                type_edge = EDGE_TYPES[e['type'][index_edge]]

                edges_list.append([e['start'][index_edge], type_edge, e['end'][index_edge]])

        if flaw_loc == '[]':
            targets.append([0])
            index_ben = index_ben + 1
        else:
            index_vul = index_vul + 1
            locations = n[n['location'].notnull()]

            for index_location,row in locations.iterrows():
                if locations['location'][index_location].split(':')[0]==flaw_loc[1:len(flaw_loc)-1]:
                    node_idx = (locations['key'][index_location])
                    targets.append([node_idx])
                    break;

        for index_node, row in n.iterrows():
            if pd.isnull(n['location'][index_node]):
                map_line.append(0)
            else:
                map_line.append(int(n['location'][index_node].split(':')[0]))


        dictionary = {"node_features": n.code.tolist(), "graph": edges_list, "targets": targets, 'map_line': map_line, 'index': [index_ben, index_vul]}

        with open("data/input/"+str(filename)+".txt", 'w') as file:
            json.dump(dictionary, file, cls=NumpyEncoder)
            file.close()

Empty node / edge CSV
Empty node / edge CSV
Empty node / edge CSV



# Train/Test/Validation split

In [47]:
raw = pd.read_csv("Datasets for VELVET/d2a_tokenized.csv")

In [28]:
ffmpeg = dict()

for commit in Repository('FFmpeg').traverse_commits():
    ffmpeg[commit.hash]=str(commit.committer_date)
    

pickle.dump(ffmpeg, open('data/projects/ffmpeg.pkl', 'wb'))

In [35]:
httpd = dict()

for commit in Repository('httpd').traverse_commits():
    httpd[commit.hash]=str(commit.committer_date)
    
pickle.dump(httpd, open('data/projects/httpd.pkl', 'wb'))

In [80]:
libtiff = dict()

for commit in Repository('libtiff').traverse_commits():
    libtiff[commit.hash]=str(commit.committer_date)
    
pickle.dump(libtiff, open('data/projects/libtiff.pkl', 'wb'))

In [38]:
nginx = dict()

for commit in Repository('nginx').traverse_commits():
    nginx[commit.hash]=str(commit.committer_date)
    
pickle.dump(nginx, open('data/projects/nginx.pkl', 'wb'))

In [40]:
openssl = dict()

for commit in Repository('openssl').traverse_commits():
    openssl[commit.hash]=str(commit.committer_date)
    
pickle.dump(openssl, open('data/projects/openssl.pkl', 'wb'))

In [82]:
raw['date'] = 0

for file in tqdm(os.listdir('data/input')):
    index = file.split('.')[0]
    project = raw['project'][int(index)]
    commit_hash = raw['version_hash'][int(index)]
    
    with open('data/projects/'+project+'.pkl', 'rb') as f:
        dictionary = pickle.load(f)
        date = dictionary[commit_hash]
        raw['date'][int(index)]= date
        
raw 

,id,label,bug_location,bug_info,bug_type,project,src,tokens,compiler_args,version_hash,prefix,suffix,date
0,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@249:1-262:2,"['void', 'FUN1', '(', 'float', '*', 'VAR1', ',...",NaN,290e7eb77bee5a54182fb3d5fb122c1e117190da,NaN,NaN,2011-12-18 14:20:14+01:00
1,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_1,1,[10],"([-oo, +oo] + `in->pitch_lag`):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@248:1-260:2,"['void', 'FUN1', '(', 'float', '*', 'VAR1', ',...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,b6b00675112b411fbb10e8b01be1d3ddc2574cf9,/*\n * adaptive and fixed codebook vector oper...,NaN,2011-12-18 13:48:44+01:00
2,ffmpeg_00254aed9cd491784527972397a55016f5c3f270_1,1,[44],"([0, +oo] × [-oo, +oo]):signed32 by call to `r...",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/twinvq.c@730:1-813:2,"['static', 'void', 'FUN1', '(', 'TwinContext',...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,5ed68178225c82f6525f962b895bce682e9ad05c,/*\n * TwinVQ decoder\n * Copyright (c) 2009 V...,\nstatic int twin_decode_frame(AVCodecContext ...,2011-10-22 15:38:05-04:00
3,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@2226:1-2345:2,"['static', 'void', 'FUN1', '(', 'AVCodecContex...",NaN,ac4b32df71bd932838043a4838b86d11e169707f,NaN,NaN,2014-04-04 04:00:11+02:00
4,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_1,1,[33],"([-oo, +oo] - 1):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@1702:1-1817:2,"['static', 'void', 'FUN1', '(', 'AVCodecContex...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,ce58ead40dd16f272ec1f127f04f69df67c7cd35,/*\n * VP8 compatible video decoder\n *\n * Co...,\nstatic void vp8_filter_mb_row(AVCodecContext...,2014-04-04 01:29:29+02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6723,openssl_ffa54528d94a1a94cbb12a803ed2889a256e32...,0,[],NaN,DEAD_STORE,openssl,crypto/asn1/a_utctm.c@16:1-22:2,"['int', 'FUN1', '(', 'struct', 'VAR1', '*', 'V...",NaN,3d0f1cb9fdd630c6c920bc97bf496538717e7705,NaN,NaN,2017-07-24 08:27:27+10:00
6724,openssl_ffa54528d94a1a94cbb12a803ed2889a256e32...,1,[32],The value written to &i (type int) is never used.,DEAD_STORE,openssl,crypto/asn1/a_utctm.c@16:1-131:2,"['int', 'FUN1', '(', 'struct', 'VAR1', '*', 'V...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,a1099821f9937717f92464056d80f2e303a73a4d,/*\n * Copyright 1995-2017 The OpenSSL Project...,\nint ASN1_UTCTIME_check(const ASN1_UTCTIME *d...,2017-07-24 07:59:40+10:00
6725,openssl_ffa7bc4ef5ec59f57a339daad0fe6983119c16...,1,[180],"([-oo, +oo] - [-oo, +oo]):signed64.",INTEGER_OVERFLOW_U5,openssl,apps/asn1pars.c@56:1-311:2,"['int', 'FUN1', '(', 'int', 'argc', ',', 'char...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,4f090f76a412a0f69f85621468bd445ea6a65af6,/*\n * Copyright 1995-2018 The OpenSSL Project...,"\nstatic int do_generate(char *genstr, const c...",2018-04-03 16:00:01+02:00
6726,openssl_ffd8fb02e505c5267d9b46914d77a10a6a7c68...,0,[],NaN,BUFFER_OVERRUN_L4,openssl,apps/req.c@1181:1-1237:2,"['static', 'int', 'FUN1', '(', 'X509_NAME', '*...",NaN,7644a9aef8932ed4d1c3f25ed776c997702982be,NaN,NaN,2015-12-16 16:14:49-05:00


In [86]:
to_delete = raw.loc[raw['date']==0].index
raw = raw.drop(to_delete)

In [90]:
raw.head()

,id,label,bug_location,bug_info,bug_type,project,src,tokens,compiler_args,version_hash,prefix,suffix,date
0,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@249:1-262:2,"['void', 'FUN1', '(', 'float', '*', 'VAR1', ',...",NaN,290e7eb77bee5a54182fb3d5fb122c1e117190da,NaN,NaN,2011-12-18 14:20:14+01:00
1,ffmpeg_000e31192eeea22671c4329d5bc9d3c9e8e5d0b6_1,1,[10],"([-oo, +oo] + `in->pitch_lag`):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/acelp_vectors.c@248:1-260:2,"['void', 'FUN1', '(', 'float', '*', 'VAR1', ',...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,b6b00675112b411fbb10e8b01be1d3ddc2574cf9,/*\n * adaptive and fixed codebook vector oper...,NaN,2011-12-18 13:48:44+01:00
2,ffmpeg_00254aed9cd491784527972397a55016f5c3f270_1,1,[44],"([0, +oo] × [-oo, +oo]):signed32 by call to `r...",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/twinvq.c@730:1-813:2,"['static', 'void', 'FUN1', '(', 'TwinContext',...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,5ed68178225c82f6525f962b895bce682e9ad05c,/*\n * TwinVQ decoder\n * Copyright (c) 2009 V...,\nstatic int twin_decode_frame(AVCodecContext ...,2011-10-22 15:38:05-04:00
3,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@2226:1-2345:2,"['static', 'void', 'FUN1', '(', 'AVCodecContex...",NaN,ac4b32df71bd932838043a4838b86d11e169707f,NaN,NaN,2014-04-04 04:00:11+02:00
4,ffmpeg_004b83ea3addb8e32893e2ca3b9fdf90bc4f27e0_1,1,[33],"([-oo, +oo] - 1):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/vp8.c@1702:1-1817:2,"['static', 'void', 'FUN1', '(', 'AVCodecContex...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,ce58ead40dd16f272ec1f127f04f69df67c7cd35,/*\n * VP8 compatible video decoder\n *\n * Co...,\nstatic void vp8_filter_mb_row(AVCodecContext...,2014-04-04 01:29:29+02:00


In [91]:
raw.to_csv("Datasets for VELVET/d2a_with_date.csv", index=False)

In [100]:
raw = raw.sort_values(by='date',ascending=True)

In [101]:
raw.shape

(6725, 13)

In [166]:
for file in raw.index:
    with open('data/input/'+str(file)+".txt", 'rb') as f:
        content = json.load(f)
        f.close()
        if not (content['targets']):
            os.remove('data/input/'+str(file)+'.txt')

In [168]:
to_delete = []

for index in raw.index:
    if str(index)+'.txt' not in os.listdir('data/input'):
        to_delete.append(index)
    
len(to_delete)

233

In [169]:
raw = raw.drop(to_delete)

In [170]:
raw

,id,label,bug_location,bug_info,bug_type,project,src,tokens,compiler_args,version_hash,prefix,suffix,date
4903,openssl_23487de988612d02837a87723a2680721e4d45...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,crypto/pem/pem_info.c@85:1-264:3,"['FUN1', '(', 'VAR1', ')', '*', 'FUN2', '(', '...",NaN,20b85fdd7644aa940e50a158a1b2c8010bb36443,NaN,NaN,1999-05-04 08:56:51+00:00
5884,openssl_9a9c83c18e2600a291a95e797e4e418cef64ec...,1,[34],"([0, +oo] + 1):unsigned32 by call to `dump_cer...",INTEGER_OVERFLOW_L5,openssl,crypto/x509/x509_obj.c@66:1-185:3,"['char', '*', 'FUN1', '(', 'X509_NAME', '*', '...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,e40b7abeed32f51f57e4578254aa1559762a8ea2,/* crypto/x509/x509_obj.c */\n/* Copyright (C)...,\n,1999-05-08 12:59:50+00:00
5883,openssl_9a9c83c18e2600a291a95e797e4e418cef64ec...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,crypto/x509/x509_obj.c@66:1-185:3,"['char', '*', 'FUN1', '(', 'X509_NAME', '*', '...",NaN,a5ab0532caf9a11f20eeb49b78ddc8e90f64a418,NaN,NaN,1999-05-08 22:46:51+00:00
5454,openssl_6b1205a6353ff4b0a71dd1e4a9032692c9af4d...,1,[15],"Offset added: [1, +oo] Size: [1, +oo] by call ...",BUFFER_OVERRUN_L4,openssl,crypto/conf/conf.c@665:1-700:3,"['static', 'CONF_VALUE', '*', 'FUN1', '(', 'LH...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,11af1a2758baefab1157755c508d7b7490155bab,/* crypto/conf/conf.c */\n/* Copyright (C) 199...,NaN,1999-06-18 18:22:38+00:00
4693,openssl_0e8e8f28dfd04afb558c7b4ab4d0462fa462dd...,1,[14],"Offset: [1, +oo] Size: [0, +oo] by call to `sk...",BUFFER_OVERRUN_L3,openssl,crypto/stack/stack.c@182:1-204:3,"['char', '*', 'FUN1', '(', 'STACK', '*', 'VAR1...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,11af1a2758baefab1157755c508d7b7490155bab,/* crypto/stack/stack.c */\n/* Copyright (C) 1...,"\nint sk_find(STACK *st, char *data)\n\t{\n\tc...",1999-06-18 18:22:38+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302,libtiff_dac5ca76fa9bf186583f8c62a193c188c5ff95...,0,[],NaN,INFERBO_ALLOC_MAY_BE_BIG,libtiff,tools/tiffcrop.c@782:1-987:4,"['static', 'int', 'FUN1', '(', 'VAR1', '*', 'V...",NaN,632f6a1a4f830f69d62fc0314b2109a2d9025963,NaN,NaN,2020-04-03 19:11:09+00:00
4153,httpd_e1977c035e4b82aa5f55fec76627076b759eeece_0,0,[],NaN,BUFFER_OVERRUN_L5,httpd,server/protocol.c@1347:1-1583:2,"['request_rec', '*', 'FUN1', '(', 'conn_rec', ...",NaN,b157ae396cd69db81fe47d25ae173e4b9e60bf8d,NaN,NaN,2020-04-17 13:07:46+00:00
4012,httpd_265b4cc0b2bc048f90df102688e1dac34a5efa79_1,1,[113],"([0, +oo] + [0, +oo]):unsigned64 by call to `a...",INTEGER_OVERFLOW_U5,httpd,server/protocol.c@1112:1-1367:2,"['FUN1', '(', 'void', ')', 'FUN2', '(', 'reque...",-DLINUX -D_REENTRANT -D_GNU_SOURCE -I/gpfs/r92...,3fce584ad134fd80d30e8a5bacc0076be119dece,/* Licensed to the Apache Software Foundation ...,\nAP_DECLARE(void) ap_get_mime_headers(request...,2020-04-21 10:18:08+00:00
4049,httpd_5ce838c20c3fa34ffefb17741c6e2e10d40550f9_1,1,[110],The value written to &cur_timeout (type long) ...,DEAD_STORE,httpd,server/protocol.c@1433:1-1601:2,"['request_rec', '*', 'FUN1', '(', 'conn_rec', ...",-DLINUX -D_REENTRANT -D_GNU_SOURCE -I/gpfs/r92...,3fce584ad134fd80d30e8a5bacc0076be119dece,/* Licensed to the Apache Software Foundation ...,\n/* if a request with a body creates a subreq...,2020-04-21 10:18:08+00:00


In [178]:
len_raw = raw.shape[0]

len_train = int(len_raw * (90/100)+0.5)
len_dev = int(len_raw * (5/100)+0.5)
len_eval = int(len_raw * (5/100))

In [179]:
print(len_train)
print(len_dev)
print(len_eval)

print(len_train+len_dev+len_eval==len_raw)

5843
325
324
True


In [180]:
train = raw[0:5843]
train

,id,label,bug_location,bug_info,bug_type,project,src,tokens,compiler_args,version_hash,prefix,suffix,date
4903,openssl_23487de988612d02837a87723a2680721e4d45...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,crypto/pem/pem_info.c@85:1-264:3,"['FUN1', '(', 'VAR1', ')', '*', 'FUN2', '(', '...",NaN,20b85fdd7644aa940e50a158a1b2c8010bb36443,NaN,NaN,1999-05-04 08:56:51+00:00
5884,openssl_9a9c83c18e2600a291a95e797e4e418cef64ec...,1,[34],"([0, +oo] + 1):unsigned32 by call to `dump_cer...",INTEGER_OVERFLOW_L5,openssl,crypto/x509/x509_obj.c@66:1-185:3,"['char', '*', 'FUN1', '(', 'X509_NAME', '*', '...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,e40b7abeed32f51f57e4578254aa1559762a8ea2,/* crypto/x509/x509_obj.c */\n/* Copyright (C)...,\n,1999-05-08 12:59:50+00:00
5883,openssl_9a9c83c18e2600a291a95e797e4e418cef64ec...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,crypto/x509/x509_obj.c@66:1-185:3,"['char', '*', 'FUN1', '(', 'X509_NAME', '*', '...",NaN,a5ab0532caf9a11f20eeb49b78ddc8e90f64a418,NaN,NaN,1999-05-08 22:46:51+00:00
5454,openssl_6b1205a6353ff4b0a71dd1e4a9032692c9af4d...,1,[15],"Offset added: [1, +oo] Size: [1, +oo] by call ...",BUFFER_OVERRUN_L4,openssl,crypto/conf/conf.c@665:1-700:3,"['static', 'CONF_VALUE', '*', 'FUN1', '(', 'LH...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,11af1a2758baefab1157755c508d7b7490155bab,/* crypto/conf/conf.c */\n/* Copyright (C) 199...,NaN,1999-06-18 18:22:38+00:00
4693,openssl_0e8e8f28dfd04afb558c7b4ab4d0462fa462dd...,1,[14],"Offset: [1, +oo] Size: [0, +oo] by call to `sk...",BUFFER_OVERRUN_L3,openssl,crypto/stack/stack.c@182:1-204:3,"['char', '*', 'FUN1', '(', 'STACK', '*', 'VAR1...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,11af1a2758baefab1157755c508d7b7490155bab,/* crypto/stack/stack.c */\n/* Copyright (C) 1...,"\nint sk_find(STACK *st, char *data)\n\t{\n\tc...",1999-06-18 18:22:38+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,ffmpeg_627bf5e8ac52ef147e9eece93d3f741508645719_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavcodec/wmv2dec.c@147:1-244:2,"['int', 'FUN1', '(', 'MpegEncContext', '*', 'V...",NaN,65e0a7c473f23f1833538ffecf53c81fe500b5e4,NaN,NaN,2017-11-18 11:57:57+01:00
5008,openssl_3120f1c976b3bfeffdf1c1b057339df900f63c...,0,[],NaN,BUFFER_OVERRUN_L4,openssl,apps/pkeyparam.c@36:1-141:2,"['int', 'FUN1', '(', 'int', 'argc', ',', 'char...",NaN,b0004708730f300a2e5c6a11c887caab50b6c42a,NaN,NaN,2017-11-20 07:20:30+01:00
6002,openssl_a6a7a6d92f3f8caa9688b8150ffcdf37686cb3...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,test/evp_extra_test.c@496:1-570:2,"['static', 'int', 'FUN1', '(', 'int', 'VAR1', ...",NaN,b0004708730f300a2e5c6a11c887caab50b6c42a,NaN,NaN,2017-11-20 07:20:30+01:00
3784,ffmpeg_f259fd58fa2172351f3d58733d08ab975220d52c_1,1,[72],"([-oo, +oo] + 1):signed32.",INTEGER_OVERFLOW_L5,ffmpeg,libavformat/dashenc.c@1048:1-1121:2,"['static', 'int', 'FUN1', '(', 'AVFormatContex...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,2d9cf3bf16b94cd9db10dabad695c69c5cff4f58,/*\n * MPEG-DASH ISO BMFF segmenter\n * Copyri...,\nstatic int dash_write_trailer(AVFormatContex...,2017-11-21 03:13:03+01:00


In [183]:
dev = raw[5843:6168]
dev

,id,label,bug_location,bug_info,bug_type,project,src,tokens,compiler_args,version_hash,prefix,suffix,date
1491,ffmpeg_60f976bc88389411b14217912745fa9718fe7dae_1,1,[10],"([0, +oo] + 1):unsigned8 by call to `ff_mqc_fl...",INTEGER_OVERFLOW_U5,ffmpeg,libavcodec/mqcenc.c@31:1-49:2,"['static', 'void', 'FUN1', '(', 'MqcState', '*...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,e8e9306b4f0017c3d2ae2a9f02136279c1a105e9,/*\n * MQ-coder encoder\n * Copyright (c) 2007...,\nstatic void renorme(MqcState *mqc)\n{\n d...,2017-11-22 00:08:35+01:00
175,ffmpeg_0df01b0b520698f7e014f6701f4bb4789d89dc3a_1,1,[22],"Offset: [1, +oo] Size: 8192 by call to `decode...",BUFFER_OVERRUN_L4,ffmpeg,libavcodec/jpeg2000dec.c@1487:1-1563:2,"['static', 'int', 'FUN1', '(', 'Jpeg2000Decode...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,e8e9306b4f0017c3d2ae2a9f02136279c1a105e9,/*\n * JPEG 2000 image decoder\n * Copyright (...,\n/* TODO: Verify dequantization for lossless ...,2017-11-22 00:08:35+01:00
968,ffmpeg_3fcb9f0fdbb894089a0b5be63dbe9dc96f276f6f_0,0,[],NaN,INTEGER_OVERFLOW_L5,ffmpeg,libavformat/mov.c@2434:1-2529:2,"['int', 'FUN1', '(', 'MOVContext', '*', 'VAR1'...",NaN,6db511a7838830f856b4664958add937a4a0d49b,NaN,NaN,2017-11-22 13:06:50+01:00
1490,ffmpeg_60f976bc88389411b14217912745fa9718fe7dae_0,0,[],NaN,INTEGER_OVERFLOW_U5,ffmpeg,libavcodec/j2kenc.c@640:1-694:2,"['static', 'void', 'FUN1', '(', 'Jpeg2000Encod...",NaN,3d5822d9cf07d08bce82903e4715658f46b01b5c,NaN,NaN,2017-11-22 13:06:50+01:00
126,ffmpeg_0a1883e1372706549c0ea5323f1e49c10cb066be_1,1,[33],"Offset: [-oo, +oo] Size: [0, +oo] by call to `...",BUFFER_OVERRUN_L5,ffmpeg,fftools/ffmpeg_hw.c@67:1-215:2,"['int', 'FUN1', '(', 'const', 'char', '*', 'VA...",-I/gpfs/r92gpfs02/suneja/infer_dataset/ffmpeg ...,3a71bcc213f223428622ac3750fe1a923f2f3ab4,/*\n * This file is part of FFmpeg.\n *\n * FF...,\nvoid hw_device_free_all(void)\n{\n int i;...,2017-11-26 21:41:19+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6272,openssl_c647238f482e448e9ec51ca412290ab7d114a6...,0,[],NaN,BUFFER_OVERRUN_L5,openssl,ssl/ssl_lib.c@2877:1-3064:2,"['SSL_CTX', '*', 'FUN1', '(', 'const', 'SSL_ME...",NaN,98732979001dbb59320803713c4c91ba40234250,NaN,NaN,2018-11-12 14:29:02+00:00
4974,openssl_2bd99edbd370aa5562012fc451604e20938f5a...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,ssl/statem/statem_lib.c@2290:1-2304:2,"['const', 'FUN1', '(', 'VAR1', ')', '*', 'FUN2...",NaN,98732979001dbb59320803713c4c91ba40234250,NaN,NaN,2018-11-12 14:29:02+00:00
6407,openssl_d93eb9057855759d7b1551415285f396db43a5...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,ssl/ssl_lib.c@3693:1-3808:2,"['SSL', '*', 'FUN1', '(', 'SSL', '*', 'VAR1', ...",NaN,98732979001dbb59320803713c4c91ba40234250,NaN,NaN,2018-11-12 14:29:02+00:00
6320,openssl_cc0216acf74c7375826b90bc2c58cd520d2478...,0,[],NaN,BUFFER_OVERRUN_L4,openssl,apps/pkeyutl.c@80:1-414:2,"['int', 'FUN1', '(', 'int', 'argc', ',', 'char...",NaN,9d1bf5f7dee88e2fc1149be16256404265765894,NaN,NaN,2018-11-18 18:52:02+01:00


In [184]:
eval = raw.iloc[-324:]
eval

,id,label,bug_location,bug_info,bug_type,project,src,tokens,compiler_args,version_hash,prefix,suffix,date
5552,openssl_76d220f8700350c1af95f2f1bb522a67aa41a1...,0,[],NaN,INTEGER_OVERFLOW_L5,openssl,crypto/err/err.c@200:1-254:2,"['static', 'void', 'FUN1', '(', 'void', ')', '...",NaN,9f15e5b911ba6053e09578f190354568e01c07d7,NaN,NaN,2018-11-24 13:08:56+01:00
6324,openssl_cc2f8eb3e804704ece60deeaf7345070a93aa3...,1,[24],"Offset: [0, +oo] Size: [0, 8388607].",BUFFER_OVERRUN_L4,openssl,crypto/bn/bn_word.c@61:1-96:2,"['BN_ULONG', 'FUN1', '(', 'BIGNUM', '*', 'VAR1...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,3da2e9c4ee45989a426ff513dc6c6250d1e460de,/*\n * Copyright 1995-2016 The OpenSSL Project...,"\nint BN_add_word(BIGNUM *a, BN_ULONG w)\n{\n ...",2018-12-05 10:33:57+00:00
5213,openssl_4ab2e7a7cf5cc5c3a55fdd9452f064bb57aacc...,1,[19],"Offset: [0, +oo] Size: [1, +oo] by call to `BN...",BUFFER_OVERRUN_L4,openssl,crypto/bn/bn_ctx.c@252:1-272:2,"['static', 'int', 'FUN1', '(', 'BN_STACK', '*'...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,3da2e9c4ee45989a426ff513dc6c6250d1e460de,/*\n * Copyright 2000-2018 The OpenSSL Project...,\nstatic unsigned int BN_STACK_pop(BN_STACK *s...,2018-12-05 10:33:57+00:00
5749,openssl_8c499916d505f27303c7ecdeec18c64252303d...,1,[22],"(2 × [1, +oo]):signed32 by call to `EC_POINT_s...",INTEGER_OVERFLOW_L5,openssl,crypto/bn/bn_sqr.c@27:1-106:2,"['int', 'FUN1', '(', 'BIGNUM', '*', 'VAR1', ',...",-I/gpfs/r92gpfs02/suneja/infer_dataset/openssl...,3da2e9c4ee45989a426ff513dc6c6250d1e460de,/*\n * Copyright 1995-2018 The OpenSSL Project...,\n/* tmp must have 2*n words */\nvoid bn_sqr_n...,2018-12-05 10:33:57+00:00
6698,openssl_fcc1a74c0017b68477ea14ebb60dd1359f6f47...,0,[],NaN,BUFFER_OVERRUN_L3,openssl,crypto/bn/bn_shift.c@107:1-151:2,"['int', 'FUN1', '(', 'BIGNUM', '*', 'VAR1', ',...",NaN,3a4a88f436ed1dd1165e0b59c1ca4a25e9e1d690,NaN,NaN,2018-12-05 10:33:57+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302,libtiff_dac5ca76fa9bf186583f8c62a193c188c5ff95...,0,[],NaN,INFERBO_ALLOC_MAY_BE_BIG,libtiff,tools/tiffcrop.c@782:1-987:4,"['static', 'int', 'FUN1', '(', 'VAR1', '*', 'V...",NaN,632f6a1a4f830f69d62fc0314b2109a2d9025963,NaN,NaN,2020-04-03 19:11:09+00:00
4153,httpd_e1977c035e4b82aa5f55fec76627076b759eeece_0,0,[],NaN,BUFFER_OVERRUN_L5,httpd,server/protocol.c@1347:1-1583:2,"['request_rec', '*', 'FUN1', '(', 'conn_rec', ...",NaN,b157ae396cd69db81fe47d25ae173e4b9e60bf8d,NaN,NaN,2020-04-17 13:07:46+00:00
4012,httpd_265b4cc0b2bc048f90df102688e1dac34a5efa79_1,1,[113],"([0, +oo] + [0, +oo]):unsigned64 by call to `a...",INTEGER_OVERFLOW_U5,httpd,server/protocol.c@1112:1-1367:2,"['FUN1', '(', 'void', ')', 'FUN2', '(', 'reque...",-DLINUX -D_REENTRANT -D_GNU_SOURCE -I/gpfs/r92...,3fce584ad134fd80d30e8a5bacc0076be119dece,/* Licensed to the Apache Software Foundation ...,\nAP_DECLARE(void) ap_get_mime_headers(request...,2020-04-21 10:18:08+00:00
4049,httpd_5ce838c20c3fa34ffefb17741c6e2e10d40550f9_1,1,[110],The value written to &cur_timeout (type long) ...,DEAD_STORE,httpd,server/protocol.c@1433:1-1601:2,"['request_rec', '*', 'FUN1', '(', 'conn_rec', ...",-DLINUX -D_REENTRANT -D_GNU_SOURCE -I/gpfs/r92...,3fce584ad134fd80d30e8a5bacc0076be119dece,/* Licensed to the Apache Software Foundation ...,\n/* if a request with a body creates a subreq...,2020-04-21 10:18:08+00:00


In [185]:
print(len(train))
print(len(dev))
print(len(eval))

5843
325
324


In [186]:
for file in train.index:
    shutil.move('data/input/'+str(file)+'.txt', 'data/dataset_d2a/train/'+str(file)+'.txt')

In [187]:
for file in dev.index:
    shutil.move('data/input/'+str(file)+'.txt', 'data/dataset_d2a/dev/'+str(file)+'.txt')

In [188]:
for file in eval.index:
    shutil.move('data/input/'+str(file)+'.txt', 'data/dataset_d2a/eval/'+str(file)+'.txt')